In [1]:
import os
from tqdm import tqdm, tqdm_notebook
import numpy as np
import pandas as pd
from scipy.misc import imread
import matplotlib.pyplot as plt
from keras.preprocessing import image
from math import *
import cv2
import keras.backend as K

import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
tf.python.control_flow_ops = control_flow_ops

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, InputLayer
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dropout
from keras.applications.vgg16 import VGG16

from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

%matplotlib inline

Using TensorFlow backend.


In [2]:
def image_generator(driving_log, data_path, normalizer=255.0, steering_shift=3., steering_max=25.):
    driving_log = driving_log.sample(frac=1).reset_index(drop=True)

    for index, row in driving_log.iterrows():
        fname  = os.path.basename(row['center'])
        fname1 = os.path.basename(row['left'])
        fname2 = os.path.basename(row['right'])
        
        img  = imread(data_path+'IMG/'+fname).astype(np.float32)
        img1 = imread(data_path+'IMG/'+fname1).astype(np.float32)
        img2 = imread(data_path+'IMG/'+fname2).astype(np.float32)
        
        yield img, np.float32(row['steering'])
        yield img1, np.float32(row['steering'])+steering_shift/steering_max
        yield img2, np.float32(row['steering'])-steering_shift/steering_max
        
        if abs(np.float32(row['steering'])) > 0.5/25.:
            yield np.fliplr(img), -np.float32(row['steering'])
        
#         if abs(np.float32(row['steering'])-steering_shift/steering_max) > 1./25.:
#             yield np.fliplr(img2), -(np.float32(row['steering'])-steering_shift/steering_max)

#         if abs(np.float32(row['steering'])+steering_shift/steering_max) > 1./25.:
#             yield np.fliplr(img1), -(np.float32(row['steering'])+steering_shift/steering_max)

def batch_generator(driving_log, data_path, preprocess_fn = None, batch_size=32, *args, **kwargs):
    num_rows = len(driving_log.index)
    train_images = np.zeros((batch_size, img_rows, img_cols, 3))
    train_steering = np.zeros((batch_size,1))
    ctr = None
    while 1:        
        for j in range(batch_size):
            # Reset generator if over bounds
            if ctr is None or ctr >= num_rows:
                ctr = 0
                images = image_generator(driving_log, data_path, *args, **kwargs)
            train_images[j], train_steering[j] = next(images)
            ctr += 1
        if preprocess_fn is not None:
            yield preprocess_fn(train_images), (train_steering+1.)/2.
        else:
            yield (train_images), (train_steering+1.)/2.

In [3]:
data_path = './driving-data/'
test_path = './test-data/'

img_rows, img_cols = 160, 320

In [4]:
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing import image

In [5]:
columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
driving_log = pd.read_csv(data_path+'driving_log_trimmed.csv', names=columns)
driving_log = driving_log.sample(frac=1).reset_index(drop=True) # Shuffle

test_log = pd.read_csv(test_path+'driving_log.csv', names=columns)
test_log = test_log.sample(frac=1).reset_index(drop=True) # Shuffle

train_data = batch_generator(driving_log, data_path, preprocess_fn=preprocess_input, batch_size=32)
val_data = batch_generator(test_log.sample(frac=0.1), test_path, preprocess_fn=preprocess_input, batch_size=32)
test_data = batch_generator(test_log.sample(frac=0.9), test_path, preprocess_fn=preprocess_input, batch_size=32)
print(len(driving_log))
print(len(test_log))

10172
1692


In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [18]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_2
1 convolution2d_95
2 batchnormalization_95
3 convolution2d_96
4 batchnormalization_96
5 convolution2d_97
6 batchnormalization_97
7 maxpooling2d_4
8 convolution2d_98
9 batchnormalization_98
10 convolution2d_99
11 batchnormalization_99
12 maxpooling2d_5
13 convolution2d_103
14 batchnormalization_103
15 convolution2d_101
16 convolution2d_104
17 batchnormalization_101
18 batchnormalization_104
19 averagepooling2d_11
20 convolution2d_100
21 convolution2d_102
22 convolution2d_105
23 convolution2d_106
24 batchnormalization_100
25 batchnormalization_102
26 batchnormalization_105
27 batchnormalization_106
28 mixed0
29 convolution2d_110
30 batchnormalization_110
31 convolution2d_108
32 convolution2d_111
33 batchnormalization_108
34 batchnormalization_111
35 averagepooling2d_12
36 convolution2d_107
37 convolution2d_109
38 convolution2d_112
39 convolution2d_113
40 batchnormalization_107
41 batchnormalization_109
42 batchnormalization_112
43 batchnormalization_113
44 mixed1
45 convolution

In [ ]:
base_model_1 = Model(input=base_model.input, output=base_model.get_layer('mixed8').output)

In [ ]:
img_path = './driving-data/IMG/center_2016_11_28_22_31_45_470.jpg'
img = imread(img_path).astype(np.float32)
x = preprocess_input(np.expand_dims(img, axis=0))

base_features = base_model_1.predict(x)
print(base_features.shape)

In [ ]:
plt.rcParams['figure.figsize'] = (10,40)
for i in range(64):
    plt.subplot(16,4,i+1)
    plt.imshow(base_features[0,:,:,i], cmap='gray')
plt.show()

In [20]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, 3))
# we chose to train the top 4 inception blocks, i.e. we will freeze
# the first (136) layers and unfreeze the rest:
for layer in base_model.layers[:114]:
   layer.trainable = False
for layer in base_model.layers[114:]:
   layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
# x = Dense(1024, activation='relu')(x)
# x = Dense(1000)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
x = Dense(100)(x)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
steering_pred = Dense(1)(x)
model = Model(input=base_model.input, output=steering_pred)

In [21]:
opt = Adam(lr=0.0001)
model.compile(optimizer='adam', loss='mae')

In [ ]:
h = model.fit_generator(train_data, validation_data = val_data,
                        samples_per_epoch = 9600,
                        nb_val_samples = 960,
                        nb_epoch=5, verbose=1)

Epoch 1/5
1696/9600 [====>.........................] - ETA: 64s - loss: 0.0885

In [ ]:
# inp = next(test_data)
test_loss = model.predict(inp[0])
# print(inp[0].shape)
# print(inp[1].shape)
# pred = model.predict(inp[0])
# print(pred.shape)
# d = abs(pred-inp[1])
# print(d)
# print(test_loss[3])
# print(test_loss)
# print(inp[1][-3,:].shape)
# plt.show()

In [ ]:
model_json = model.to_json()
model_name = 'model_i1'
with open(model_name+'.json', "w") as json_file:
    json_file.write(model_json)

model.save_weights(model_name+'.h5')